In [1]:
# These are the imports that we will need to get started. This list will grow over time as we want to do other things besides just explore...
import pandas as pd
from dfply import *

In [2]:
# Read in the dataset. This filepath is mine. This is the full dataset. 
train = pd.read_csv("E:\\ProjectData\\ERAU-REU\\Project-Cyber-Security\\NSL-KDD\\KDDTrain+.txt",header=None)
test = pd.read_csv("E:\\ProjectData\\ERAU-REU\\Project-Cyber-Security\\NSL-KDD\\KDDTest+.txt",header=None)
train.columns = ['duration', 'protocol_type', 'service', 'flag', 'src_bytes',
'dst_bytes', 'land', 'wrong_fragment', 'urgent', 'hot',
'num_failed_logins', 'logged_in', 'num_compromised', 'root_shell',
'su_attempted', 'num_root', 'num_file_creations', 'num_shells',
'num_access_files', 'num_outbound_cmds', 'is_host_login',
'is_guest_login', 'count', 'srv_count', 'serror_rate',
'srv_serror_rate', 'rerror_rate', 'srv_rerror_rate', 'same_srv_rate',
'diff_srv_rate', 'srv_diff_host_rate', 'dst_host_count',
'dst_host_srv_count', 'dst_host_same_srv_rate','dst_host_diff_srv_rate', 'dst_host_same_src_port_rate',
'dst_host_srv_diff_host_rate', 'dst_host_serror_rate',
'dst_host_srv_serror_rate', 'dst_host_rerror_rate',
'dst_host_srv_rerror_rate', 'classification', 'difficulty_level']

test.columns = ['duration', 'protocol_type', 'service', 'flag', 'src_bytes',
'dst_bytes', 'land', 'wrong_fragment', 'urgent', 'hot',
'num_failed_logins', 'logged_in', 'num_compromised', 'root_shell',
'su_attempted', 'num_root', 'num_file_creations', 'num_shells',
'num_access_files', 'num_outbound_cmds', 'is_host_login',
'is_guest_login', 'count', 'srv_count', 'serror_rate',
'srv_serror_rate', 'rerror_rate', 'srv_rerror_rate', 'same_srv_rate',
'diff_srv_rate', 'srv_diff_host_rate', 'dst_host_count',
'dst_host_srv_count', 'dst_host_same_srv_rate','dst_host_diff_srv_rate', 'dst_host_same_src_port_rate',
'dst_host_srv_diff_host_rate', 'dst_host_serror_rate',
'dst_host_srv_serror_rate', 'dst_host_rerror_rate',
'dst_host_srv_rerror_rate', 'classification', 'difficulty_level']

In [3]:
train >>= mutate(anomaly = case_when([train.classification == 'normal','normal'],
[train.classification != 'normal','anomaly']))

test >>= mutate(anomaly = case_when([test.classification == 'normal','normal'],
[test.classification != 'normal','anomaly']))

In [4]:
# This is general information about the structure of the dataset.
print(train.info())
print(test.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 125973 entries, 0 to 125972
Data columns (total 44 columns):
 #   Column                       Non-Null Count   Dtype  
---  ------                       --------------   -----  
 0   duration                     125973 non-null  int64  
 1   protocol_type                125973 non-null  object 
 2   service                      125973 non-null  object 
 3   flag                         125973 non-null  object 
 4   src_bytes                    125973 non-null  int64  
 5   dst_bytes                    125973 non-null  int64  
 6   land                         125973 non-null  int64  
 7   wrong_fragment               125973 non-null  int64  
 8   urgent                       125973 non-null  int64  
 9   hot                          125973 non-null  int64  
 10  num_failed_logins            125973 non-null  int64  
 11  logged_in                    125973 non-null  int64  
 12  num_compromised              125973 non-null  int64  
 13 

In [5]:
# Here, we are literally just viewing the first couple lines of the dataset to help us see how it looks in tabular form. 
train.protocol_type = train.protocol_type.astype('category')
train.service = train.protocol_type.astype('category')
train.flag = train.protocol_type.astype('category')
train.classification = train.protocol_type.astype('category')
train.anomaly = train.anomaly.astype('category')

test.protocol_type = test.protocol_type.astype('category')
test.service = test.protocol_type.astype('category')
test.flag = test.protocol_type.astype('category')
test.classification = test.protocol_type.astype('category')
test.anomaly = test.anomaly.astype('category')

In [6]:
print(train.info())
print(test.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 125973 entries, 0 to 125972
Data columns (total 44 columns):
 #   Column                       Non-Null Count   Dtype   
---  ------                       --------------   -----   
 0   duration                     125973 non-null  int64   
 1   protocol_type                125973 non-null  category
 2   service                      125973 non-null  category
 3   flag                         125973 non-null  category
 4   src_bytes                    125973 non-null  int64   
 5   dst_bytes                    125973 non-null  int64   
 6   land                         125973 non-null  int64   
 7   wrong_fragment               125973 non-null  int64   
 8   urgent                       125973 non-null  int64   
 9   hot                          125973 non-null  int64   
 10  num_failed_logins            125973 non-null  int64   
 11  logged_in                    125973 non-null  int64   
 12  num_compromised              125973 non-null